# Recomender system

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.sparse import csr_matrix

In [ ]:
%load_ext nb_black

Part 1)

In [2]:
MV_users = pd.read_csv('../Data/users.csv')
MV_movies = pd.read_csv('../Data/movies.csv')
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [8]:
from sklearn.decomposition import TruncatedSVD

In [45]:
train_pivoted = train.pivot(index='uID',columns='mID',values='rating').fillna(0)
train_csr = csr_matrix(train_pivoted.values)

In [46]:
svd = TruncatedSVD(n_components=5)

In [47]:
svd.fit(train_csr,)

TruncatedSVD(n_components=5)

In [48]:
trained_user_preferences = svd.transform(train_pivoted)

In [49]:
trained_movie_features = svd.components_

In [63]:
def predict_svd(uid, mid, uID_index, mID_index, trained_user_preferences, trained_movie_features):
    try:
        return np.dot(
            trained_user_preferences[list(uID_index).index(uid),:],
            trained_movie_features[:,list(mID_index).index(mid)]
        )
    except ValueError:
        return 3
    

In [61]:
train['pred'] = train.apply(lambda row: predict_svd(
    row['uID'], 
    row['mID'],
    train_pivoted.index,
    train_pivoted.columns,
    trained_user_preferences,
    trained_movie_features
), axis=1)

In [65]:
test['pred'] =test.apply(lambda row: predict_svd(
    row['uID'], 
    row['mID'],
    train_pivoted.index,
    train_pivoted.columns,
    trained_user_preferences,
    trained_movie_features
), axis=1)

KeyboardInterrupt: 

In [ ]:
train